# Batch-wise Structure Relaxation

In this tutorial, we show how to use the ``TorchStructureLBFGS``. It enables relaxation of structures in a batch-wise manner, i.e. it optimizes multiple structures in parallel. This is particularly useful, when many relatively similar structures (--> similar time until convergence) should be relaxed while requiring possibly short simulation time.

In [1]:
import torch
from ase.io import read

import schnetpack as spk
from schnetpack import properties
from schnetpack.interfaces.ase_interface import AtomsConverter
from schnetpack.interfaces.batchwise_optimizer import ASEBatchwiseLBFGS

First, we load the force field model that provides the forces for the relaxation process. Furthermore, we define the atoms converter, which is used to convert ase Atoms objects to SchNetPack input.

In [2]:
model_path = "../../tests/testdata/md_ethanol.model"

# set device
device = torch.device("cuda")

# load model
model = torch.load(model_path, map_location=device)

# define neighbor list
cutoff = model.representation.cutoff.item()
nbh_list=spk.transform.MatScipyNeighborList(cutoff=cutoff)

# build atoms converter
atoms_converter = AtomsConverter(
    neighbor_list=nbh_list,
    device=device,
)

Subsequently, we load the batch of initial structures utilizing ASE (supports xyz, db and more).

In [3]:
input_structure_file = "../../tests/testdata/md_ethanol.xyz"

# load initial structures
ats = read(input_structure_file, index=":")

For some systems it helps to fix the positions of certain atoms during the relaxation. This can be achieved by providing a mask of boolean entries to ``ASEBatchwiseLBFGS``. The mask is a list of $n_\text{atoms}$ entries, indicating atoms, which positions are fixed during the relaxation. Here, we do not fix any atoms. Hence, the mask only contains ``True``.

In [4]:
# define structure mask for optimization (True for fixed, False for non-fixed)
n_atoms = len(ats[0].get_atomic_numbers())
single_structure_mask = [False for _ in range(n_atoms)]
# expand mask by number of input structures (fixed atoms are equivalent for all input structures)
mask = single_structure_mask * len(ats)

Finally, we run the optimization:

In [5]:
# Initialize optimizer
optimizer = ASEBatchwiseLBFGS(
    model=model,
    atoms=ats,
    converter=atoms_converter,
    #fixed_atoms_mask=mask,
    trajectory=".relax_traj",
    energy_unit="kcal/mol",
    position_unit="Ang",
)

# run optimization
optimizer.run(fmax=0.0005, steps=1000)

                   Step     Time         fmax
ASEBatchwiseLBFGS:    0 12:01:16       1.4290
ASEBatchwiseLBFGS:   24 12:01:16       0.0004


True

In [13]:
# get list of optimized structures and properties
opt_atoms, opt_props = optimizer.get_relaxation_results()

for oatoms in opt_atoms:
    print(oatoms.get_positions())
    
print(opt_props)

[[-4.92642021  1.53844273 -0.06455463]
 [-3.41313696  1.45339704 -0.13812941]
 [-5.23321629  2.29324794  0.6733253 ]
 [-5.35420847  0.5697878   0.2302338 ]
 [-5.34598112  1.81654835 -1.04170418]
 [-2.99841595  1.18484378  0.85213369]
 [-2.99016571  2.43538523 -0.42366371]
 [-3.07344508  0.45518184 -1.11438727]
 [-2.10454273  0.39966059 -1.16255355]]
{'energy': tensor([-97209.5078], device='cuda:0', grad_fn=<AsStridedBackward0>), 'forces': tensor([[-0.0038,  0.0008,  0.0007],
        [-0.0088,  0.0022,  0.0020],
        [ 0.0044, -0.0020, -0.0020],
        [ 0.0039,  0.0039, -0.0023],
        [ 0.0038, -0.0022,  0.0040],
        [-0.0017,  0.0041,  0.0006],
        [-0.0017,  0.0008,  0.0041],
        [ 0.0026, -0.0048, -0.0047],
        [ 0.0014, -0.0027, -0.0026]], device='cuda:0')}
